# Issue Hospital Credentials
## Role: NHS Trust

In order to initialise the system into the correct configuration, the NHS Trust (this notebook), must connect with each of the three hospitals and issue them with a credential that attests to them being a Hospital under the trusts authority.

One the hospitals have these credentials, they can use them as many times as they wish.

## 1. Initialise a Controller for the NHS Trust Agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8022
ADMIN_URL = "http://nhstrust-agent:8021"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [2]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'RiUq77qGfmFmAHxj1NQcSY', 'verkey': 'EUJ88fdb32eFKwCaWueL3idpXQdra4myur2Zaqh2YDht', 'posture': 'wallet_only'}


## 3. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [3]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "RiUq77qGfmFmAHxj1NQcSY": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by RiUq77qGfmFmAHxj1NQcSY to the ledger with role ENDORSER"}}'}
200


## 4. Accept the Transaction Author Agreement

In [4]:
response = await agent_controller.ledger.get_taa()
TAA_text = response['result']['taa_record']['text']
TAA_version = response['result']['taa_record']['version']
TAA_digest = response['result']['taa_record']['digest']


taa_record = {
            "version": TAA_version,
            "mechanism": "service_agreement",
            'text': TAA_text
}

response = await agent_controller.ledger.accept_taa(taa_record)
## Will return {} if successful
print(response)

{}


## 5. Assign Public DID

In [5]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'RiUq77qGfmFmAHxj1NQcSY', 'verkey': 'EUJ88fdb32eFKwCaWueL3idpXQdra4myur2Zaqh2YDht', 'posture': 'public'}}


In [6]:

# Or feel free to define your own schema

# Define you schema name - must be unique on the ledger
# schema_name = "NHS Hosptial"
# # Can version the schema if you wish to update it
# schema_version = "0.0.1"
# # Define any list of attributes you wish to include in your schema
# attributes = ["name"]

# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# schema_id = response["schema_id"]
# print(schema_id)

## 6. Write Credential Definition for NHS Hosptial Schema

The schema has already been written to the ledger here

In [7]:
schema_id = "BvRKFu1fdzGdJzQcKb8e8p:2:NHS Hosptial:0.0.1"
response = await agent_controller.definitions.write_cred_def(schema_id)

## 7. Store Definition and Schema ID

In [8]:
cred_def_id = response["credential_definition_id"]
print(cred_def_id)
%store cred_def_id
%store schema_id

RiUq77qGfmFmAHxj1NQcSY:3:CL:156569:default
Stored 'cred_def_id' (str)
Stored 'schema_id' (str)


## 8. Terminate the controller


In [9]:
await agent_controller.terminate()

## NHS Trust Inititialised

Continue to [Part 3 - Issue Hospital Credentials](http://localhost:8888/notebooks/Part%203%20-%20Issue%20Hospital%20Credentials.ipynb)
